In [147]:
stop_words_portuguese = ['de',
 'a',
 'o',
 'que',
 'e',
 'do',
 'da',
 'em',
 'um',
 'para',
 'com',
 'não',
 'uma',
 'os',
 'no',
 'se',
 'na',
 'por',
 'mais',
 'as',
 'dos',
 'como',
 'mas',
 'ao',
 'ele',
 'das',
 'à',
 'seu',
 'sua',
 'ou',
 'quando',
 'muito',
 'nos',
 'já',
 'eu',
 'também',
 'só',
 'pelo',
 'pela',
 'até',
 'isso',
 'ela',
 'entre',
 'depois',
 'sem',
 'mesmo',
 'aos',
 'seus',
 'quem',
 'nas',
 'me',
 'esse',
 'eles',
 'você',
 'essa',
 'num',
 'nem',
 'suas',
 'meu',
 'às',
 'minha',
 'numa',
 'pelos',
 'elas',
 'qual',
 'nós',
 'lhe',
 'deles',
 'essas',
 'esses',
 'pelas',
 'este',
 'dele',
 'tu',
 'te',
 'vocês',
 'vos',
 'lhes',
 'meus',
 'minhas',
 'teu',
 'tua',
 'teus',
 'tuas',
 'nosso',
 'nossa',
 'nossos',
 'nossas',
 'dela',
 'delas',
 'esta',
 'estes',
 'estas',
 'aquele',
 'aquela',
 'aqueles',
 'aquelas',
 'isto',
 'aquilo',
 'estou',
 'está',
 'estamos',
 'estão',
 'estive',
 'esteve',
 'estivemos',
 'estiveram',
 'estava',
 'estávamos',
 'estavam',
 'estivera',
 'estivéramos',
 'esteja',
 'estejamos',
 'estejam',
 'estivesse',
 'estivéssemos',
 'estivessem',
 'estiver',
 'estivermos',
 'estiverem',
 'hei',
 'há',
 'havemos',
 'hão',
 'houve',
 'houvemos',
 'houveram',
 'houvera',
 'houvéramos',
 'haja',
 'hajamos',
 'hajam',
 'houvesse',
 'houvéssemos',
 'houvessem',
 'houver',
 'houvermos',
 'houverem',
 'houverei',
 'houverá',
 'houveremos',
 'houverão',
 'houveria',
 'houveríamos',
 'houveriam',
 'sou',
 'somos',
 'são',
 'era',
 'éramos',
 'eram',
 'fui',
 'foi',
 'fomos',
 'foram',
 'fora',
 'fôramos',
 'seja',
 'sejamos',
 'sejam',
 'fosse',
 'fôssemos',
 'fossem',
 'for',
 'formos',
 'forem',
 'serei',
 'será',
 'seremos',
 'serão',
 'seria',
 'seríamos',
 'seriam',
 'tenho',
 'tem',
 'temos',
 'tém',
 'tinha',
 'tínhamos',
 'tinham',
 'tive',
 'teve',
 'tivemos',
 'tiveram',
 'tivera',
 'tivéramos',
 'tenha',
 'tenhamos',
 'tenham',
 'tivesse',
 'tivéssemos',
 'tivessem',
 'tiver',
 'tivermos',
 'tiverem',
 'terei',
 'terá',
 'teremos',
 'terão',
 'teria',
 'teríamos',
 'teriam']

In [187]:
import re
import spacy
from string import punctuation


nlp = spacy.load('pt_core_news_sm')

PATTERN = r"""\w+-+\w+|\w+|[!'"#$%&\*\+,\./:;\?]{1}"""


class TextNLP:
    
    def __init__(self, text):
        assert isinstance(text, str), "'A classe deve ser inicializada com uma string sendo passada por parâmetro'"
        
        if text == '':
            raise ValueError('A string passada não pode estar vazia')
        
        self.text = text
        
        
    def __iter__(self):
        
        return (match.group() for match in re.finditer(PATTERN, self.text))
    
    
    def __repr__(self):
        tokens = self.words
        
        return "Texto: %s\nTokens: %s\nNum Tokens: %d" % (self.text, tokens, len(tokens))
    
    
    @property
    def words(self):
            
        return list(self.__iter__())
    
    
    def words_preprocessing(self, stop_words_remove=False, punctuation_remove=False):
        
        tokens = list(self.__iter__())
        
        if stop_words_remove:
            tokens = [tok for tok in tokens if tok not in stop_words_portuguese]
        
        if punctuation_remove:
            tokens = [tok for tok in tokens if tok not in punctuation]
            
        return tokens
    
    
    @property
    def sents(self):
        doc = nlp(self.text)
                
        return [sent for sent in doc.sents]

In [188]:
w = TextNLP('Hoje é terça-feira, e o lobo é mau! E nós também somos! Somos? Será que somos...')
print(w.words_preprocessing())

['Hoje', 'é', 'terça-feira', ',', 'e', 'o', 'lobo', 'é', 'mau', '!', 'E', 'nós', 'também', 'somos', '!', 'Somos', '?', 'Será', 'que', 'somos', '.', '.', '.']


In [190]:
w

Texto: Hoje é terça-feira, e o lobo é mau! E nós também somos! Somos? Será que somos...
Tokens: ['Hoje', 'é', 'terça-feira', ',', 'e', 'o', 'lobo', 'é', 'mau', '!', 'E', 'nós', 'também', 'somos', '!', 'Somos', '?', 'Será', 'que', 'somos', '.', '.', '.']
Num Tokens: 23

In [189]:
w.words

['Hoje',
 'é',
 'terça-feira',
 ',',
 'e',
 'o',
 'lobo',
 'é',
 'mau',
 '!',
 'E',
 'nós',
 'também',
 'somos',
 '!',
 'Somos',
 '?',
 'Será',
 'que',
 'somos',
 '.',
 '.',
 '.']

In [191]:
for item in w.words:
    print(item)

Hoje
é
terça-feira
,
e
o
lobo
é
mau
!
E
nós
também
somos
!
Somos
?
Será
que
somos
.
.
.


In [192]:
for sent in w.sents:
    print(sent)

Hoje é terça-feira, e o lobo é mau!
E nós também somos!
Somos?
Será que somos...
